In [11]:
import numpy as np
from google.colab import drive
import glob
from collections import Counter
import pandas as pd
import easydict
import shutil
from pathlib import Path

import matplotlib.pyplot as plt
import copy, pickle, os, time
import argparse
from sklearn.model_selection import train_test_split

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
args = easydict.EasyDict(
    {
    "size_dataset": 150,
    "original_dataset_path": '/content/drive/MyDrive/Articulo/dataset10k/originalData/',
    "selected_dataset_path": '/content/drive/MyDrive/Articulo/dataset10k/selectData300dt/',
    "path_model" : '/content/drive/MyDrive/Miguel UNSA/'
    })

In [13]:
# Function to select random rows for each class
def sampling_k_elements(group, k=args.size_dataset): 
    if len(group) < k:
        return group
    return group.sample(k)

In [14]:
# Function that reads folder where the source data is and converts it to PD
def read_to_pandas(path_folder):
  datas={'link':[],'class':[]}
  # root_dir needs a trailing slash (i.e. /root/dir/)
  for filename in glob.iglob(path_folder + '**/*', recursive=True):
    if filename.endswith(".jpeg") or filename.endswith(".jpg") or filename.endswith(".png"): 
      datas['link'].append(filename)
      datas['class'].append(filename.split()[0].split('/')[8])
  print("Tatal distribution data:",Counter(datas['class']))
  return pd.DataFrame(datas)

In [15]:
def balance_data(pd_data):
  return pd_data.groupby('class').apply(sampling_k_elements).reset_index(drop=True)

In [16]:
def concatenate_split_data(pd_balance):
  train, val = train_test_split(pd_balance, test_size=0.25, random_state=42)
  train['split']='train'
  val['split']='val'
  all_data = pd.concat([train, val], ignore_index=True, sort=False)
  return all_data

In [17]:
# Function that copies selected data to specific folder
def copy_files_selected(copy_root_path,pd_data):
  dynamic_path=copy_root_path
  for index, row in pd_data.iterrows():
    type_=row['split']
    link_=row['link']
    class_=row['class']
    path_image=link_.split('/')[-1]
    shutil.copy(link_, dynamic_path+type_+'/'+class_+'/'+path_image)

In [18]:
pd_all_data=read_to_pandas(args.original_dataset_path)
pd_balance=balance_data(pd_all_data)
final_data=concatenate_split_data(pd_balance)
print(final_data['class'].value_counts())
print("Total: ",final_data['class'].size)

Tatal distribution data: Counter({'non': 4341, 'covid': 1300})
covid    150
non      150
Name: class, dtype: int64
Total:  300


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
copy_files_selected(args.selected_dataset_path, final_data)